# Import original data - Merzy

In [3]:
import pandas as pd

In [4]:
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 21)

In [4]:
merzy_paths = ['Shopee_data/merzy/merzy.xlsx', 'Shopee_data/merzy/merzy 1.xlsx', 'Shopee_data/merzy/merzy 2.xlsx', 
               'Shopee_data/merzy/merzy 3.xlsx', 'Shopee_data/merzy/merzy 4.xlsx', 'Shopee_data/merzy/merzy 5.xlsx', 'Shopee_data/merzy/merzy 6.xlsx']
dfs = []
for path in merzy_paths:
    df = pd.read_excel(path)
    dfs.append(df)

In [5]:
merzy_df = pd.concat(dfs, ignore_index=True)

In [6]:
#merzy_df.to_excel("merzy_data_og.xlsx", index=False)

In [14]:
df_merzy = pd.read_excel("merzy_data_og.xlsx")
df_merzy.shape

(19837, 21)

In [15]:
df_merzy.head(1)

,Unnamed: 0,Unnamed: 1,Author,Date,Rating,Rating Delivery Service,Rating Product Quality,Rating Seller Service,Review,Version,Oversea Shopping,Region,Picture included,Photos,Videos,Profile Url,Review Url,Unnamed: 17,Unnamed: 18,Unnamed: 19,(view source)
0,1,NaN,t*****2,2023-10-17 01:13:00,5,5,5,5,"Son đẹp, giao hàng nhanh, đóng gói sản phẩm chắc chắn, sản phẩm như hình ❤️❤️",🆕 CM4,no,vn,no,NaN,NaN,Rating Total: 2388; No Stars: 14 reviews; *: 10 reviews; **: 25 reviews; ***: 83 reviews; ****: ...,https://shopee.vn/shop/12822129,https://shopee.vn/p-i.193207976.23357766479,NaN,NaN,NaN


### Drop irrelevant columns

In [18]:
df_merzy.drop(columns=['Unnamed: 0', 'Rating Delivery Service', 'Rating Product Quality', 'Rating Seller Service', 'Region', 'Photos', 
                       'Videos', 'Profile Url', 'Review Url', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', '(view source)'], inplace=True)

In [21]:
df_merzy.rename(columns={'Unnamed: 1': 'Seller', 'Author': 'Customer'}, inplace=True)

In [24]:
df_merzy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19837 entries, 0 to 19836
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Seller            0 non-null      float64       
 1   Customer          19757 non-null  object        
 2   Date              19837 non-null  datetime64[ns]
 3   Rating            19837 non-null  int64         
 4   Review            16587 non-null  object        
 5   Version           19837 non-null  object        
 6   Oversea Shopping  19837 non-null  object        
 7   Picture included  19837 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 1.2+ MB


In [59]:
df_merzy['Seller'].fillna('Merzy Official Store', inplace=True)

In [78]:
df_merzy['Seller'].iloc[4690:(4690+3030)] = df_merzy['Seller'].iloc[4690:(4690+3030)].str.replace('Merzy Official Store', 'Thế Giới Skin Food', regex=True)

In [79]:
df_merzy_dup = df_merzy.loc[df_merzy.duplicated() == True]
df_merzy_dup.shape

(36, 8)

# Roughly preprocessed data for comment analysis

In [135]:
df_merzy_cleaned = df_merzy.loc[df_merzy['Review'].notna() == True].reset_index()
df_merzy_cleaned.drop(columns=['index'], inplace=True)

In [136]:
df_merzy_cleaned.shape

(16587, 8)

In [138]:
df_merzy_cleaned.head()

,Seller,Customer,Date,Rating,Review,Version,Oversea Shopping,Picture included
0,Merzy Official Store,t*****2,2023-10-17 01:13:00,5,"Son đẹp, giao hàng nhanh, đóng gói sản phẩm chắc chắn, sản phẩm như hình ❤️❤️",🆕 CM4,no,no
1,Merzy Official Store,nguyentrang.2901,2023-10-18 03:31:00,5,"Son mùi hơi kì tí 🥲 , màu nghiên về cam đất tí , độ bám vừa không quá lâu_x000D_\nhttps://cf.sho...",🆕 CM6,no,yes
2,Merzy Official Store,quyentruongdo,2023-10-18 03:40:00,5,"Chất son mịn, mùi thơm trái cây_x000D_\nhttps://cf.shopee.vn/file/vn-11134103-7r98o-lmzjhxxd1xi0...",🆕 CM5,no,yes
3,Merzy Official Store,phucan23678,2023-10-18 04:03:00,5,Trời ơi màu xinh xỉu \nChất son mịn như nhungg :33,🆕 CM6,no,no
4,Merzy Official Store,hoaan_nguyen,2023-10-18 06:11:00,5,"Màu xinh vãi chất son thì khỏi bàn r mấy bà ơi chốt trên live nên rẻ, tui sợ cm3 sẽ base hồng nh...",🆕 CM3,no,yes


### Fill in missing customer names

In [161]:
list_na_index = df_merzy_cleaned.loc[df_merzy_cleaned['Customer'].notna() == False].index.values

In [163]:
for i in list_na_index:
    df_merzy_cleaned.at[i, 'Customer'] = f'Customer #{i}'

### Drop duplicated reviews

In [171]:
df_merzy_cleaned.drop_duplicates(keep='first', inplace=True)

### Data Overview

In [172]:
df_merzy_cleaned.describe()

,Rating
count,16575.000000
mean,4.911131
std,0.430390
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


In [173]:
df_merzy_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16575 entries, 0 to 16586
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Seller            16575 non-null  object        
 1   Customer          16575 non-null  object        
 2   Date              16575 non-null  datetime64[ns]
 3   Rating            16575 non-null  int64         
 4   Review            16575 non-null  object        
 5   Version           16575 non-null  object        
 6   Oversea Shopping  16575 non-null  object        
 7   Picture included  16575 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.1+ MB


In [174]:
# df_merzy_cleaned.to_excel('raw_cleaned_merzy.xlsx', index=False)

# Import original data - Focallure

In [5]:
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 21)

In [23]:
foc_paths = ['Shopee_data/focallure/foc.xlsx', 'Shopee_data/focallure/foc1.xlsx', 'Shopee_data/focallure/foc2.xlsx', 'Shopee_data/focallure/foc3.xlsx',
           'Shopee_data/focallure/foc4.xlsx', 'Shopee_data/focallure/foc5.xlsx', 'Shopee_data/focallure/foc6.xlsx', 'Shopee_data/focallure/foc7.xlsx',
           'Shopee_data/focallure/foc8.xlsx', 'Shopee_data/focallure/foc9.xlsx', 'Shopee_data/focallure/foc10.xlsx', 'Shopee_data/focallure/foc11.xlsx',
           'Shopee_data/focallure/foc12.xlsx', 'Shopee_data/focallure/foc13.xlsx', 'Shopee_data/focallure/foc14.xlsx', 'Shopee_data/focallure/foc15.xlsx',
           'Shopee_data/focallure/foc16.xlsx']
dfs = []
for path in foc_paths:
    df = pd.read_excel(path)
    dfs.append(df)

In [24]:
foc_df = pd.concat(dfs, ignore_index=True)

In [10]:
# foc_df.to_excel('focallure_data_og.xlsx', index=False)

In [25]:
foc_df = pd.read_excel('focallure_data_og.xlsx')

In [26]:
foc_df.head(1)

,Unnamed: 0,Unnamed: 1,Author,Date,Rating,Rating Delivery Service,Rating Product Quality,Rating Seller Service,Review,Version,Oversea Shopping,Region,Picture included,Photos,Videos,(view source),Seller
0,1,NaN,diutranghong276,2023-11-20 13:21:14,5,5,5,5,"Công dụng:makeup\nĐộ bền màu:siêu bền luôn í\n\nson phải goin là siêu bền luôn, lớp tint màu xinh lắm lun rất rcm mn mua ạaaa\nhttps://cf.shopee.vn/file/vn-11134103-7r98o-lob7tnazdx1nff\nhttps://c...",#RD02,no,vn,yes,"https://cf.shopee.vn/file/vn-11134103-7r98o-lob7tnazdx1nff, https://cf.shopee.vn/file/vn-11134103-7r98o-lob7tnaztczb28",NaN,view review,NaN


### Drop irrelevant columns amd duplicated comments

In [27]:
foc_df.drop(columns=['Unnamed: 0', 'Rating Delivery Service', 'Rating Product Quality', 'Rating Seller Service', 'Region', 'Photos', 
                       'Videos', '(view source)', 'Seller'], inplace=True)

In [28]:
foc_df.rename(columns={'Unnamed: 1': 'Seller', 'Author': 'Customer'}, inplace=True)

In [31]:
foc_df['Seller'].fillna('Focallure Official', inplace=True)

In [35]:
foc_cleaned = foc_df.loc[foc_df['Review'].notna() == True].reset_index()
foc_cleaned.drop(columns=['index'], inplace=True)

In [37]:
foc_cleaned.drop_duplicates(keep='first', inplace=True)

### Fill in missing customer names

In [38]:
list_na_index = foc_cleaned.loc[foc_cleaned['Customer'].notna() == False].index.values
for i in list_na_index:
    foc_cleaned.at[i, 'Customer'] = f'Customer #{i}'

In [48]:
foc_cleaned.shape

(26505, 8)

In [38]:
foc_cleaned.head(1)

,Seller,Customer,Date,Rating,Review,Version,Oversea Shopping,Picture included
0,Focallure Official,diutranghong276,2023-11-20 13:21:14,5,"Công dụng:makeup\nĐộ bền màu:siêu bền luôn í\n\nson phải goin là siêu bền luôn, lớp tint màu xinh lắm lun rất rcm mn mua ạaaa\nhttps://cf.shopee.vn/file/vn-11134103-7r98o-lob7tnazdx1nff\nhttps://c...",#RD02,no,yes


In [39]:
foc_cleaned.to_excel('raw_cleaned_foc.xlsx', index=False)